In [1]:
!pip install torch transformers flask beautifulsoup4 requests nltk


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install accelerate -U


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

model_name = 'Xenova/gpt-3.5-turbo'
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT3_5Tokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


OSError: Xenova/gpt-3.5-turbo does not appear to have a file named config.json. Checkout 'https://huggingface.co/Xenova/gpt-3.5-turbo/main' for available files.

In [4]:
import pandas as pd
import re
import unicodedata
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder

# Clean text function
def clean_text(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = re.sub(r"[\r\n\t\\]+", " ", text)  # Replace newlines, tabs, and backslashes
    text = re.sub(r"[\u2019\u2018]", "'", text)  # Normalize apostrophes
    text = re.sub(r"[\u201c\u201d]", '"', text)  # Normalize quotation marks
    text = re.sub(r"[^\x00-\x7F]+", " ", text)  # Remove non-ASCII characters
    return ' '.join(text.split())  # Remove extra spaces

# Load and preprocess the dataset
df = pd.read_json('Psychology-10K.json')
df['input'] = df['input'].apply(lambda x: clean_text(x) if isinstance(x, str) else None)
df['output'] = df['output'].apply(lambda x: clean_text(x) if isinstance(x, str) else None)
df.dropna(subset=['input', 'output'], inplace=True)  # Remove rows with None values

# Label Encoding
label_encoder = LabelEncoder()
df['output'] = label_encoder.fit_transform(df['output'])

# Instantiate tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class SimpleTextDataset(Dataset):
    def __init__(self, tokenizer, input_texts, output_texts, max_length=512):
        self.input_texts = input_texts
        self.output_texts = output_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        source = self.input_texts[idx]
        target = self.output_texts[idx]  # These are now integer labels

        # Tokenize source
        source_encodings = tokenizer.encode_plus(
            source,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': source_encodings['input_ids'].squeeze(),  # Remove extra dimension
            'attention_mask': source_encodings['attention_mask'].squeeze(),
            'labels': torch.tensor(target)  # Ensure labels are tensors
        }

# Prepare dataset
input_data = df['input'].tolist()
output_data = df['output'].tolist()
text_dataset = SimpleTextDataset(tokenizer, input_data, output_data)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=10,
    logging_steps=1,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize model with the correct number of labels
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=text_dataset,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [5]:
# Train the model
trainer.train()

  0%|          | 0/1970 [00:00<?, ?it/s]

{'loss': 9.1559, 'grad_norm': 6.911580562591553, 'learning_rate': 4.9974619289340105e-05, 'epoch': 0.0}
{'loss': 9.1464, 'grad_norm': 6.759867191314697, 'learning_rate': 4.994923857868021e-05, 'epoch': 0.0}
{'loss': 9.2759, 'grad_norm': 7.027420520782471, 'learning_rate': 4.992385786802031e-05, 'epoch': 0.0}
{'loss': 9.242, 'grad_norm': 7.115017890930176, 'learning_rate': 4.9898477157360406e-05, 'epoch': 0.0}
{'loss': 9.6245, 'grad_norm': 7.3831682205200195, 'learning_rate': 4.9873096446700515e-05, 'epoch': 0.01}
{'loss': 9.3084, 'grad_norm': 9.334020614624023, 'learning_rate': 4.984771573604061e-05, 'epoch': 0.01}
{'loss': 9.4124, 'grad_norm': 6.55010986328125, 'learning_rate': 4.9822335025380714e-05, 'epoch': 0.01}
{'loss': 9.3925, 'grad_norm': 7.4751410484313965, 'learning_rate': 4.9796954314720816e-05, 'epoch': 0.01}
{'loss': 9.3879, 'grad_norm': 7.787557601928711, 'learning_rate': 4.977157360406092e-05, 'epoch': 0.01}
{'loss': 9.1696, 'grad_norm': 6.729978084564209, 'learning_rate

KeyboardInterrupt: 

In [1]:
def predict_response(input_text, model, tokenizer):
    # Preprocess and tokenize the input text
    encoded_input = tokenizer.encode_plus(
        input_text,
        max_length=512,  # Ensure the length matches what was used during training
        truncation=True,
        padding='max_length',
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Move the tensors to the same device as the model
    encoded_input = {key: tensor.to(model.device) for key, tensor in encoded_input.items()}

    # Make prediction
    with torch.no_grad():
        output = model(**encoded_input)

    # Retrieve the probabilities (logits) and find the predicted class
    logits = output.logits
    predicted_class_index = logits.argmax(-1).item()

    # If you used LabelEncoder during preprocessing, you need to reverse that encoding:
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_label

# Example usage
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Make sure the model is on the right device
input_text = "i feel lonely"
response = predict_response(input_text, model, tokenizer)
print("Predicted Response:", response)

NameError: name 'model' is not defined